In [1]:
import pandas as pd
import glob
import os

In [2]:
folder_path = "C:/Users/Katharina/Desktop/Weiterbildung/Bootcamp/Bootcamp/Final_project/data"

# Liste aller CSV-Dateien, die mit "player_individuell" anfangen
csv_files = glob.glob(os.path.join(folder_path, "player_list_individuell*.csv"))

# Alle CSV-Dateien einlesen und in einer Liste speichern
dfs = [pd.read_csv(file, sep=';') for file in csv_files]

# Falls du alle DataFrames zu einem großen zusammenführen möchtest:
df_combined = pd.concat(dfs, ignore_index=True)


In [3]:
df2 = df_combined.dropna(subset=['Player.@LastName'])

In [4]:
os.chdir('C:/Users/Katharina/Desktop/Weiterbildung/Bootcamp/Bootcamp/Final_project/data')
DatS = pd.read_csv('DatenML_V1.csv', sep=';')

In [ ]:
test = pd.merge(DatS[["FirstName", "LastName"]], df2[['Player.@FederationCode', 'Player.@FirstName', 'Player.@Gender',
       'Player.@LastName','Player.@Height','Player.@BeachHighBlock', 'Player.@BeachHighJump',
       'Player.@BeachPosition', 'Player.@BeachYearBegin', 'Player.@Birthdate',
       'Player.@Handedness']], left_on=["FirstName", "LastName"], right_on=["Player.@FirstName", "Player.@LastName"], how="left")

In [6]:
#welche spieler fehlen?

playerMissing = test[test['Player.@FirstName'].isna()].drop_duplicates()

In [10]:
#csv beach_teams --> da sind die nummern drin --> damit scrapen
os.chdir('C:/Users/Katharina/Desktop/Weiterbildung/Bootcamp/Bootcamp/Final_project/data')

BeachTeams = pd.read_csv('beach_teams.csv', sep=',')

In [ ]:
BeachTeams = BeachTeams.dropna(subset=['NoPlayer1'])


In [16]:
BeachTeams[["Player1", "Player2"]] = BeachTeams["Name"].str.split("/", expand=True, n=1)

In [17]:
BeachTeams2 = BeachTeams.drop_duplicates(subset='Name')

In [20]:
test2 = pd.merge(playerMissing[["FirstName", "LastName"]], BeachTeams2[['Player1', 'NoPlayer1']],how='left', left_on='LastName', right_on='Player1')

In [22]:
import pandas as pd
import numpy as np


# Funktion zum Ermitteln des modalen Werts; 
# wird der Mode nicht ermittelt (leere Serie), gibt sie np.nan zurück.
def get_mode(series):
    modes = series.mode()
    return modes[0] if not modes.empty else np.nan

# Gruppiere nach FirstName und LastName und bestimme den häufigsten Wert in NoPlayer
mode_df = test2.groupby(["FirstName", "LastName"])["NoPlayer1"].agg(get_mode).reset_index()
mode_df = mode_df.rename(columns={"NoPlayer1": "ModeNoPlayer1"})

# Merge den modalen Wert zurück in das Original-DataFrame
test2 = test2.merge(mode_df, on=["FirstName", "LastName"], how="left")

In [ ]:
test3 = test2.copy()
test3 = test3.drop_duplicates(subset=['ModeNoPlayer1'])
test3.dropna()

In [25]:
test3.to_csv('MissingPlayer_listID.csv', index=False, sep=';')

In [33]:
#Variante zwei
test2_A = pd.merge(playerMissing[["FirstName", "LastName"]], BeachTeams2[['Player2', 'NoPlayer2']],how='left', left_on='LastName', right_on='Player2')

# Gruppiere nach FirstName und LastName und bestimme den häufigsten Wert in NoPlayer
mode_df = test2_A.groupby(["FirstName", "LastName"])["NoPlayer2"].agg(get_mode).reset_index()
mode_df = mode_df.rename(columns={"NoPlayer2": "ModeNoPlayer2"})

# Merge den modalen Wert zurück in das Original-DataFrame
test2_B = test2_A.merge(mode_df, on=["FirstName", "LastName"], how="left")

#Duplikate entfernen
test3_A = test2_B.copy()
test3_A = test3_A.drop_duplicates(subset=['ModeNoPlayer2'])
test3_A = test3_A.dropna()

In [34]:
test3_A.to_csv('MissingPlayer_listID_V2.csv', index=False, sep=';')

In [27]:
#restliche spieler gescrapt
MissPl = pd.read_csv('player_list_Missing0705.csv', sep=';')

In [35]:
#Teil2 der Missing gescrapt
MissPl2 = pd.read_csv('player_list_Missing0805.csv', sep=';')

In [ ]:
DatMiss = pd.concat([MissPl, MissPl2],axis=0)
DatMiss1 = DatMiss.drop_duplicates(subset=['Player.@No'])

#-> Vergleich mit PlayerMissing: FirstName, LastName

In [40]:
#Prüfen, welche spieler jetzt gefunden worden sind
MISS_DF = pd.merge(DatMiss1, playerMissing[["FirstName", 'LastName']], left_on=["Player.@FirstName", 'Player.@LastName'], 
                   right_on=["FirstName", 'LastName'], how="inner")

In [46]:
#Zusammenfügen mit anderer Spielerliste -> df2 (Drop 'NoData.@id') MissDF (drop 'FirstName', 'LastName')
PLayerList = df2.drop(columns='NoData.@id')
PlayerMiss = MISS_DF.drop(columns=['FirstName', 'LastName'])

In [47]:
#Listen zusammenfügen

Player_fin = pd.concat([PLayerList, PlayerMiss], axis=0)

In [48]:
Player_fin.to_csv('Player_properties_final.csv', index=False, sep=';',)